In [1]:
import torch 
import torch.nn as nn
import torchvision.transforms.functional as F 

# Model

In [28]:
class DoubleCov(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels,
                out_channels,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, img):
        print('img', img.size())
        out = self.conv(img)
        print('out', out.size())
        return self.conv(img)

In [29]:
class UNet(nn.Module):
    def __init__(
        self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super().__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNet
        for f in features:
            self.downs.append(DoubleCov(in_channels, f))
            in_channels = f

        # Up part of UNet
        for f in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(f * 2, f, kernel_size=2, stride=2)
            )

            self.ups.append(DoubleCov(f*2, f))

        self.bottleneck = DoubleCov(features[-1], features[-1] * 2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # Down part
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            print(x.size())
            x = self.pool(x)

        # Bottle neck part
        x = self.bottleneck(x)

        # Up part
        # because it from bottom to top, we need to reversed the list
        skip_connections = list(reversed(skip_connections))
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = F.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)